<h1>LONDON WALKING TOUR</h1>

In [1]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

In [2]:
# import all the libraries that will be used in this notebook.

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim
#from geopy.distance import great_circle

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium # map rendering library

# library to cluster with DBSCAN and Kmeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [3]:
# London latitude and longitude, obtained by Google Search
latitude = 51.5074
longitude = -0.1278

In [4]:
# My Foursquare credentials
CLIENT_ID = '540VZYVEHH3C3YOU2VWXYFW1YWZI320IACMYEFSZTDZEJQVA' # your Foursquare ID
CLIENT_SECRET = 'RDYDGJSTBSNIVIDQ1I0NXPS3BYTKZEZ1D2AIYGYDJEBC2VTE' # your Foursquare Secret
VERSION = '20191024' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 540VZYVEHH3C3YOU2VWXYFW1YWZI320IACMYEFSZTDZEJQVA
CLIENT_SECRET:RDYDGJSTBSNIVIDQ1I0NXPS3BYTKZEZ1D2AIYGYDJEBC2VTE


In [5]:
# create the API request URL for Museums
radius=4000
LIMIT=100
urlm = 'https://api.foursquare.com/v2/venues/explore?&categoryId=4bf58dd8d48988d181941735&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

# create the API request URL for Landmarks
radius=4000
LIMIT=100
urll = 'https://api.foursquare.com/v2/venues/explore?&categoryId=4bf58dd8d48988d12d941735&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)


In [6]:
# get the results
resultsm = requests.get(urlm).json()
resultsl = requests.get(urll).json()

In [7]:
# put the museum results into dataframe
venuesm = resultsm['response']['groups'][0]['items']
    
vdfm = json_normalize(venuesm) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.location.lat', 'venue.location.lng','venue.location.distance']
vdfm =vdfm.loc[:, filtered_columns]


# clean columns
vdfm.columns = [col.split(".")[-1] for col in vdfm.columns]

vdfm.sort_values('distance', inplace=True)
print(vdfm.shape)

(73, 4)


In [8]:
# put the landmark results into dataframe
venuesl = resultsl['response']['groups'][0]['items']
    
vdfl = json_normalize(venuesl) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.location.lat', 'venue.location.lng','venue.location.distance']
vdfl =vdfl.loc[:, filtered_columns]

# clean columns
vdfl.columns = [col.split(".")[-1] for col in vdfl.columns]

vdfl.sort_values('distance', inplace=True)
print(vdfl.shape)

(100, 4)


In [9]:
# combine the two dataframes into one.
vdf = vdfm.append(vdfl)
vdf.sort_values("distance", inplace=True)
vdf.reset_index(inplace=True)
vdf.drop(columns='index', inplace=True)
vdf.drop_duplicates(inplace=True)
vdf.reset_index(inplace=True)
vdf.drop(columns='index', inplace=True)
print(vdf.shape)
vdf

(168, 4)


,name,lat,lng,distance
0,Nelson's Column,51.507744,-0.127931,39
1,Trafalgar Square,51.507987,-0.128048,67
2,Admiralty Arch,51.506826,-0.128789,93
3,The Royal Marines Monument,51.506652,-0.128918,113
4,Sainsbury Wing National Gallery,51.508384,-0.129001,137
5,National Gallery,51.508876,-0.128478,170
6,Northumbland Ave,51.508399,-0.125903,172
7,Benjamin Franklin House,51.507667,-0.124963,198
8,National Portrait Gallery,51.509438,-0.128032,227
9,The Household Cavalry Museum,51.504869,-0.126594,293


In [10]:
# check out the locations in a folium map
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lon, name in zip(vdf['lat'], vdf['lng'], vdf['name']):
    label=folium.Popup(str(name), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        color='red',
        popup=label,
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_london)
map_london

In [11]:
# cluster the locations using DBSCAN
# credit to Professor Geoff Boeing from USC for the method
# Prof Boeing's paper = url: https://osf.io/preprints/socarxiv/nzhdc/
coords = vdf.as_matrix(columns=['lat', 'lng'])
kms_per_radian = 6371.0088 # Python calculates geo distance in radians
epsilon = 0.26 / kms_per_radian # Epsilon is max distance the location is from each other within a cluster.

# we set the minimum samples equal to 5 so the information center can serve at least 5 venues
db = DBSCAN(eps=epsilon, min_samples=5, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
print('Number of clusters: {}'.format(num_clusters))

Number of clusters: 10


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [12]:
vdf

,name,lat,lng,distance
0,Nelson's Column,51.507744,-0.127931,39
1,Trafalgar Square,51.507987,-0.128048,67
2,Admiralty Arch,51.506826,-0.128789,93
3,The Royal Marines Monument,51.506652,-0.128918,113
4,Sainsbury Wing National Gallery,51.508384,-0.129001,137
5,National Gallery,51.508876,-0.128478,170
6,Northumbland Ave,51.508399,-0.125903,172
7,Benjamin Franklin House,51.507667,-0.124963,198
8,National Portrait Gallery,51.509438,-0.128032,227
9,The Household Cavalry Museum,51.504869,-0.126594,293


In [13]:
# add clustering labels
#vdf.drop('Cluster Labels', axis=1, inplace=True)
vdf.insert(0, 'Cluster Labels', cluster_labels)

In [14]:
# Show the clusters in a map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, poi, cluster in zip(vdf['lat'], vdf['lng'], vdf['name'], vdf['Cluster Labels']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [15]:
# since cluster 0 has so many venues, we will split this into two clusters using KMeans
vcdf = vdf.loc[vdf['Cluster Labels'] == 0]
coords = vcdf.as_matrix(columns=['lat', 'lng'])
Kclusters = 2
kmeans = KMeans(n_clusters=Kclusters, random_state=0).fit(np.radians(coords))
cluster_labels = kmeans.labels_
clusters = pd.Series([coords[cluster_labels == n] for n in range(Kclusters)])

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()


In [16]:
# add clustering labels
vcdf.drop('Cluster Labels', axis=1, inplace=True) 
vcdf.insert(0, 'Cluster Labels', cluster_labels+(num_clusters-1))

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [17]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, poi, cluster in zip(vcdf['lat'], vcdf['lng'], vcdf['name'], vcdf['Cluster Labels']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [18]:
# we will clean up the dataframe.  first, we remove the outliers and cluster 0 from the first dataframe.  Then append the separated cluster 0 back to the first df
# we store this in a new dataframe
indexNames = vdf[ (vdf['Cluster Labels'] < 1)  ].index
vdf.drop(indexNames , inplace=True)
vndf = vdf.append(vcdf)
vndf.sort_values(['Cluster Labels','distance'], inplace=True)
vndf.reset_index(inplace=True)
vndf.drop(columns='index', inplace=True)

In [19]:
vndf

,Cluster Labels,name,lat,lng,distance
0,1,Canada Gate,51.502454,-0.141389,1090
1,1,Queen Victoria Memorial,51.501837,-0.140875,1097
2,1,Australia Gate,51.501074,-0.140004,1100
3,1,Canadian War Memorial,51.502882,-0.142501,1136
4,1,Buckingham Palace Gate,51.501543,-0.141288,1139
5,1,The Queen's Gallery,51.499798,-0.142282,1312
6,2,HQS Wellington,51.510679,-0.112214,1139
7,2,Two Temple Place,51.511523,-0.112236,1171
8,2,Submariners Monument,51.511040,-0.111563,1195
9,2,City of London Dragon,51.511050,-0.110966,1235


In [20]:
# create the final map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters+Kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, poi, cluster in zip(vndf['lat'], vndf['lng'], vndf['name'], vndf['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1: Near Buckingham Palace

In [21]:
vndf.loc[vndf['Cluster Labels'] == 1]

,Cluster Labels,name,lat,lng,distance
0,1,Canada Gate,51.502454,-0.141389,1090
1,1,Queen Victoria Memorial,51.501837,-0.140875,1097
2,1,Australia Gate,51.501074,-0.140004,1100
3,1,Canadian War Memorial,51.502882,-0.142501,1136
4,1,Buckingham Palace Gate,51.501543,-0.141288,1139
5,1,The Queen's Gallery,51.499798,-0.142282,1312


### Cluster 2: Near Temple

In [22]:
vndf.loc[vndf['Cluster Labels'] == 2]

,Cluster Labels,name,lat,lng,distance
6,2,HQS Wellington,51.510679,-0.112214,1139
7,2,Two Temple Place,51.511523,-0.112236,1171
8,2,Submariners Monument,51.511040,-0.111563,1195
9,2,City of London Dragon,51.511050,-0.110966,1235
10,2,Middle Temple,51.512441,-0.111619,1253
11,2,Temple Bar Monument,51.513657,-0.111859,1305
12,2,HMS President (1918),51.510608,-0.108240,1401


### Cluster 3: Near Wellington Arch

In [23]:
vndf.loc[vndf['Cluster Labels'] == 3]

,Cluster Labels,name,lat,lng,distance
13,3,The Vault at Hard Rock Cafe,51.503826,-0.148858,1512
14,3,RAF Bomber Command Memorial,51.503292,-0.148986,1537
15,3,Memorial Monument,51.502603,-0.148820,1551
16,3,The Memorial Gates,51.502460,-0.149069,1572
17,3,Wellington Arch,51.502539,-0.150821,1684
18,3,Wellington Monument,51.504282,-0.151706,1692
19,3,Apsley House,51.503316,-0.151563,1708
20,3,Australian War Memorial,51.502127,-0.151504,1744


### Cluster 4: South Bank

In [24]:
vndf.loc[vndf['Cluster Labels'] == 4]

,Cluster Labels,name,lat,lng,distance
21,4,Blavatnik Building,51.507447,-0.099928,1931
22,4,Turbine Hall,51.507813,-0.099454,1964
23,4,Tate Modern,51.507704,-0.099456,1964
24,4,Shakespeare's Globe Exhibition,51.508076,-0.097455,2103
25,4,Shakespeare's Globe Theatre,51.508121,-0.096962,2138


### Cluster 5: Near Museum of Zoology

In [25]:
vndf.loc[vndf['Cluster Labels'] == 5]

,Cluster Labels,name,lat,lng,distance
26,5,Grant Museum of Zoology,51.523870,-0.134275,1887
27,5,Jeremy Bentham Auto Icon,51.524447,-0.133100,1932
28,5,Wellcome Collection,51.525861,-0.133907,2098
29,5,Magic Circle Headquarters,51.526516,-0.135294,2190
30,5,Euston Railway War Memorial,51.526990,-0.132554,2205


### Cluster 6: Near St. Paul's Cathedral

In [26]:
vndf.loc[vndf['Cluster Labels'] == 6]

,Cluster Labels,name,lat,lng,distance
31,6,Central Criminal Court (Old Bailey),51.515687,-0.101902,2017
32,6,College of Arms,51.512007,-0.098475,2095
33,6,Temple Bar,51.514095,-0.099440,2101
34,6,Paternoster Square Column,51.514431,-0.099175,2132
35,6,St Paul's Cathedral,51.513877,-0.098426,2159


### Cluster 7: South Kensington

In [27]:
vndf.loc[vndf['Cluster Labels'] == 7]

,Cluster Labels,name,lat,lng,distance
36,7,Victoria and Albert Museum (V&A),51.496705,-0.172440,3314
37,7,Science Museum Launch Pad,51.497559,-0.174256,3400
38,7,Museum of the History of Medicine,51.497433,-0.174316,3408
39,7,Science Museum,51.497358,-0.174434,3419
40,7,Exploring Space,51.497289,-0.175418,3486
41,7,Making The Modern World,51.497145,-0.176656,3572
42,7,Natural History Museum,51.496521,-0.176532,3587
43,7,Earth Hall,51.496268,-0.176409,3589
44,7,"Who Am I? Science Museum, Wellcome Wing",51.497231,-0.177084,3597
45,7,Science Museum Gallery of Flight,51.497189,-0.177837,3648


### Cluster 8: Near Tower of London / Tower Bridge

In [28]:
vndf.loc[vndf['Cluster Labels'] == 8]

,Cluster Labels,name,lat,lng,distance
47,8,HMS Belfast (HMS Belfast (C35)),51.506599,-0.081507,3208
48,8,Tower of London Riverside Walk,51.507861,-0.078621,3407
49,8,City Hall (Greater London Authority),51.505035,-0.078633,3416
50,8,Traitors' Gate,51.507685,-0.076897,3527
51,8,Bloody Tower,51.507804,-0.076707,3540
52,8,Tower of London,51.508248,-0.076261,3572
53,8,The White Tower,51.508076,-0.076049,3586
54,8,Tower Bridge Exhibition,51.505962,-0.075308,3640
55,8,Tower Bridge,51.505550,-0.075338,3640


### Cluster 9: Westminster

In [29]:
vndf.loc[vndf['Cluster Labels'] == 9]

,Cluster Labels,name,lat,lng,distance
56,9,Benjamin Franklin House,51.507667,-0.124963,198
57,9,The Household Cavalry Museum,51.504869,-0.126594,293
58,9,Banqueting House,51.504645,-0.126249,324
59,9,Guards Memorial,51.504595,-0.129438,332
60,9,Whitehall Gardens,51.506354,-0.122900,358
61,9,Air Force Memorial,51.505522,-0.123088,387
62,9,Monty Field Marshal Montgomery of Alamein,51.503533,-0.125973,448
63,9,Women of World War II,51.503451,-0.126008,456
64,9,The Cenotaph,51.502570,-0.126047,551
65,9,Battle of Britain Memorial,51.503094,-0.123443,566


### Cluster 10: Near Trafalgar Square, Picadilly Circus and Leicester Square

In [30]:
vndf.loc[vndf['Cluster Labels'] ==10]

,Cluster Labels,name,lat,lng,distance
73,10,Nelson's Column,51.507744,-0.127931,39
74,10,Trafalgar Square,51.507987,-0.128048,67
75,10,Admiralty Arch,51.506826,-0.128789,93
76,10,The Royal Marines Monument,51.506652,-0.128918,113
77,10,Sainsbury Wing National Gallery,51.508384,-0.129001,137
78,10,National Gallery,51.508876,-0.128478,170
79,10,Northumbland Ave,51.508399,-0.125903,172
80,10,National Portrait Gallery,51.509438,-0.128032,227
81,10,Duke of York Monument,51.506051,-0.131521,298
82,10,Waterloo Place,51.507235,-0.132530,328
